<a href="https://colab.research.google.com/github/emilycalvert/scripts_cite/blob/main/verficar_tallando.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alistar entorno.

In [1]:
!pip install numpy
!pip install geopy
!pip install pandas
!pip install pyproj
!pip install geemap
!pip install timedelta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for timedelta: filename=timedelta-2020.12.3-py3-none-any.whl size=1554 sha256=3ad89a2c60b14ca5120cff153993c0eac3a7fd98bb467a418ceb099bc5639e00
  Stored in directory: /root/.cache/pip/wheels/be/c0/1f/843bd73679dc7dfb26758c4effc6e273d3eebf2da2a35082b1
Successfully built timedelta


In [2]:
import ee
import time
import geemap
import datetime
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pyproj import Transformer, Proj, transform

# **1. Autorizar e inicializar Earth Engine**
#### _Autenticar e inicializar la API de Google Earth Engine (GEE)._




- Conecte e inicialice con tu cuenta de earth engine y proyecto.

In [8]:
ee.Authenticate(auth_mode="notebook")
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=fd3e35kzXno1EFeI0n8YSqUMh6Yq5OfiffQT7OsJJ3s&tc=fuRUB2c_-8z-iJgG2uNmNLa577cdHBWCMCJ3qZHv64U&cc=3itEWCEgLXa24U7acqbLTBJ4dSaeO0T6_OyUdlnQ0eo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnDUT8wpgLJIGGeq8cF4BK8DAOrxEvXnJE6yk-wVy6X2ltnJeUgaDA

Successfully saved authorization token.


# **2. Cargar CSV**

#### _Cargue el archivo CSV que contiene las coordenadas y analícelo para crear puntos._

- Exporta tu hoja de cálculo con los vértices como csv. La integridad y uniformidad del documento original es importante. Si tiene dificultades en este paso, consulte el curso [aquí.](https://www.youtube.com/watch?v=uGx0PHD6o9M)

- Al usar este cuaderno en colab. Habrá un panel a la izquierda. Si no estás en los archivos. Haga clic en el icono de la carpeta donde los botones están al ras de la pantalla. Ahora donde dice "Archivos", haz clic en el ícono de archivo con una flecha en el medio y sube tus vértices. Alternativamente, puede arrastrar y soltar en el panel de archivos.

In [57]:
# @title
csv_viaje = 'imazacoord_NOppi.csv'  # Una vez que haya cargado su archivo, en esta línea cambie el nombre por el nombre del archivo que cargó con la extensión.
df = pd.read_csv(csv_viaje)

In [62]:
# @title
df.head()

,NCONTRA,NOMTIT,NUMRUC,RESAPR,ESTE,NORTE,VERTICE,latitud,longitud
0,16-IQU/C-J-105-04,INDUSTRIAL MADERERA ZAPOTE S.A.,20103979529,RSD N° 019-2012-GRL-GGR-PRMRFFS-DER-SDPM,643560.25,9530613.00,1,-4.245528,-73.706444
1,16-IQU/C-J-105-04,INDUSTRIAL MADERERA ZAPOTE S.A.,20103979529,RSD N° 019-2012-GRL-GGR-PRMRFFS-DER-SDPM,643560.25,9527597.60,2,-4.272801,-73.706398
2,16-IQU/C-J-105-04,INDUSTRIAL MADERERA ZAPOTE S.A.,20103979529,RSD N° 019-2012-GRL-GGR-PRMRFFS-DER-SDPM,641579.82,9528512.95,3,-4.264552,-73.724254
3,16-IQU/C-J-105-04,INDUSTRIAL MADERERA ZAPOTE S.A.,20103979529,RSD N° 019-2012-GRL-GGR-PRMRFFS-DER-SDPM,640464.65,9528511.51,4,-4.264581,-73.734301
4,16-IQU/C-J-105-04,INDUSTRIAL MADERERA ZAPOTE S.A.,20103979529,RSD N° 019-2012-GRL-GGR-PRMRFFS-DER-SDPM,638560.25,9526722.71,5,-4.280789,-73.751432


In [61]:
# Aquí necesitamos proyectar desde UTM 18S a WGS84 (el CRS utilizado por Earth Engine).
transformer = Transformer.from_crs("epsg:32718", "epsg:4326", always_xy=True)
'''
Tenga en cuenta que si sus coordenadas no están en UTM 18S,
deberá consultar la documentación de pyproj y ajustar "epsg:32718"
al epsg que utilizan sus coordenadas. Aquí está el sitio donde
puedes encontrar esa información:
https://pyproj4.github.io/pyproj/stable/api/crs/crs.html
https://epsg.io/
'''
def utm_to_latlon(easting, northing):
    # Dado que transformador.transform(este, norte) ya se alinea con (x, y) => (ESTE, NORTE),
    # y desea regresar (latitud, longitud), asegúrese de capturar la salida correctamente.
    lon, lat = transformer.transform(easting, northing)
    return lat, lon  # Esto garantiza que la devolución esté en formato (latitud, longitud).

# Assuming 'df' is your DataFrame
df['latitud'], df['longitud'] = zip(*df.apply(lambda row: utm_to_latlon(row['ESTE'], row['NORTE']), axis=1)) # Donde dice 'ESTE' y 'NORTE', esto coincide de manera muy idéntica con los nombres de las columnas de latitud y longitud en sus datos.

#### Procesando

In [63]:
coordenadas = df[['latitud', 'longitud']].values.tolist()

In [64]:
puntos = [ee.Feature(ee.Geometry.Point([lon, lat])) for lat, lon in coordenadas]
puntos_fc = ee.FeatureCollection(puntos)

In [65]:
polygon = ee.Geometry.Polygon([[[lon, lat] for lat, lon in coordenadas]])

In [240]:
centroid = polygon.centroid(maxError=1)

#### Modificar

In [241]:
print(centroid.getInfo())

{'type': 'Point', 'coordinates': [-73.72766753816298, -4.349379692587443]}


In [239]:
centro= (-73.72766753816298, -4.349379692587443) # Aquí pegue las coordenadas impresas en la celda de arriba para definir el centro de su parcela.

# **3. Obtener imágenes de Landsat:**

#### _Utilice la API de Earth Engine para consultar imágenes de Landsat 8 para las fechas y la cobertura de nubes especificadas._

In [69]:
antes = '2023-01-06'
'''
Establezca esto en una fecha anterior a la emisión del permiso.
Encontrará la primera imagen de satélite en esa fecha o antes que
cumpla con sus requisitos de cobertura de nubes.
'''
despues = '2024-01-06'
'''
Esta fecha es posterior a la cosecha y encontrará
una imagen para la fecha más cercana pero no anterior
a la cosecha que cumpla con el umbral de nubosidad.
'''
limite_coberatura_nubes = 15  # Establezca esto en su umbral máximo de cobertura de nubes.

In [85]:
# @title
# Imagen antes de la emisión del permiso.
# Un día, en un futuro lejano, Landsat 8 podría retirarse por 9. En este caso, será necesario corregir el script.
landsat1 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
           .filterDate('1900-01-01', antes) \
           .filterBounds(puntos_fc) \
           .filter(ee.Filter.lt('CLOUD_COVER', limite_coberatura_nubes)) \
           .sort('CLOUD_COVER', True) \
           .first()

ndvi1 = landsat1.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

In [86]:
# @title
ndvi_antes = ndvi1.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=polygon,
    scale=30,
    maxPixels=1e9
)

ndvi_antes = ndvi_antes.get('NDVI').getInfo()

print('NDVI Antes:', ndvi_antes)

NDVI Antes: 0.4134755698772038


In [71]:
# @title
# Aquí está la foto después de la cosecha.
landsat2 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
           .filterDate('1900-01-01', despues) \
           .filterBounds(puntos_fc) \
           .filter(ee.Filter.lt('CLOUD_COVER', limite_coberatura_nubes)) \
           .sort('CLOUD_COVER', False) \
           .first()

ndvi2 = landsat2.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

In [87]:
# @title
ndvi_despues = ndvi2.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=polygon,
    scale=30,
    maxPixels=1e9
)

ndvi_despues = ndvi_despues.get('NDVI').getInfo()

print('NDVI Despues:', ndvi_despues)


NDVI Despues: 0.4144632519332896


#### Imagenes

In [72]:
landsat1

In [73]:
  ndvi1

In [74]:
landsat2

In [75]:
ndvi2

#### Estilo

In [233]:
polygonFeature = ee.Feature(polygon)

polygonCollection = ee.FeatureCollection([polygonFeature])

poly_style = polygonCollection.style(
    color='FFCC00',
    width=2,
    fillColor='00000000',
    lineType='solid'
)

In [227]:
vizParams = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0,
  'max': 30000,
}

## **Antes**

In [242]:
# @title
# Aquí vemos la trama previa a la emisión del permiso.
# https://colab.research.google.com/github/giswqs/gee-tutorials/blob/master/Image/image_visualization.ipynb#scrollTo=dIqOCYLbo0SF


Map = geemap.Map()


Map.centerObject(polygon, 11)
Map.addLayer(landsat1, vizParams, 'Antes Permiso')
Map.addLayer(poly_style, {}, 'Parcela')

Map

Map(center=[-4.349379692587443, -73.72766753816298], controls=(WidgetControl(options=['position', 'transparent…

# **4. Calcular NDVI:**
#### _Calcular y enmascarar áreas donde se excede el umbral._

In [208]:
ndvi_diff = ndvi2.subtract(ndvi1).rename('Change')
threshold = 0.26414
'''
Es importante monitorear para asegurarse de que el umbral dado
refleje con precisión el área cosechada.
Normalmente, un área deforestada estaría en el rango de 0,1 a 0,2
para el umbral del NDVI. Es posible que deba ajustar el script y
probarlo un poco desde aquí para encontrar las áreas deforestadas
para su parcela, ya que el área y las tendencias generales del NDVI
pueden afectar esto.
'''
diff_mask = ndvi_diff.gt(threshold)

In [209]:
# @title
pixelArea = ee.Image.pixelArea().mask(diff_mask)

totalArea = pixelArea.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=polygon,
    scale=30,
    maxPixels=1e9
).get('area').getInfo()

print(f"NDVI Cambio de Area: {totalArea} metricos cubiertos")

NDVI Cambio de Area: 891.5328979492188 metricos cubiertos


In [180]:
# @title
vectorizedAndSimplified = diff_mask.updateMask(diff_mask).reduceToVectors(
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=30,
    maxPixels=1e9
).map(lambda feature: ee.Feature(feature.geometry().simplify(maxError=ee.ErrorMargin(1))))

def assign_names_and_calculate_areas(feature, index):
    name = ee.String('Polygon_').cat(ee.Number(index).add(1).format())
    area = feature.geometry().area(maxError=ee.ErrorMargin(1))
    return feature.set({'name': name, 'area_m2': area})


featuresWithInfo = ee.FeatureCollection(indices.map(lambda index: assign_names_and_calculate_areas(ee.Feature(vectorizedAndSimplified.toList(vectorizedAndSimplified.size()).get(index)), index)))

def print_feature_info(feature):
    feature = ee.Feature(feature)
    name = feature.get('name').getInfo()
    area = feature.get('area_m2').getInfo()
    vertices = feature.geometry().coordinates().getInfo()
    print(f"Name: {name}, Area (sq meters): {area}, Vertices: {vertices}")

featureList = featuresWithInfo.toList(featuresWithInfo.size())

# **5. Visualizar**
#### _Muestra los polígonos identificados sobre la imagen Landsat más reciente._

In [217]:
# @title
for i in range(featureList.size().getInfo()):
    print_feature_info(featureList.get(i))

Name: Polygon_1.0, Area (sq meters): 904.4070791251461, Vertices: [[[-73.99049630504322, -4.560261210300778], [-73.99049592618773, -4.560532570076004], [-73.99022551544667, -4.560532189819145], [-73.99022589440996, -4.560260830070868], [-73.99049630504322, -4.560261210300778]]]
Name: Polygon_2.0, Area (sq meters): 1808.780246598804, Vertices: [[[[-73.98872774226417, -5.0275378833193285], [-73.9887273236672, -5.027809239240782], [-73.98845672927457, -5.027808819395168], [-73.98845714798831, -5.02753746349168], [-73.98872774226417, -5.0275378833193285]]], [[[-73.98872774226417, -5.0275378833193285], [-73.98872816083417, -5.027266527388891], [-73.9889987550202, -5.027266947072809], [-73.988998336558, -5.027538303021212], [-73.98872774226417, -5.0275378833193285]]]]
Name: Polygon_3.0, Area (sq meters): 904.3895017402504, Vertices: [[[-73.98791218828208, -5.029978826150306], [-73.98791176913716, -5.030250181990912], [-73.98764117381926, -5.030249761606309], [-73.98764159307198, -5.029978405

In [238]:
# @title
styledPolygons = vectorizedAndSimplified.style(
    color='FFCC00',
    width=.75,
    fillColor='00000000',
    lineType='solid'
)

Map = geemap.Map()

Map.centerObject(polygon, 11)

Map.addLayer(landsat2, vizParams, 'Despues Tallando')

Map.addLayer(poly_style, {}, 'Styled Polygon')

Map.addLayer(styledPolygons, {}, 'Mask Outline')

Map

Map(center=[-4.349379692587443, -73.72766753816298], controls=(WidgetControl(options=['position', 'transparent…